# Lab 2: Sequence Models

In this lab, you will implement a **Hidden Markov Model (HMM)**. HMMs specify a **joint** probability over **observations** and **hidden states**. 

This is what we will do today:

1. **Estimate** a simple HMM model from training data (supervised learning)
2. Find the *best* sequence of hidden states for a given sequence of observations (we define "best" in 2 different ways!)


------

### Rules
* The lab exercises should be made in **groups of two people**.

* The assignment should submitted to **Blackboard** as `.ipynb`. Only **one submission per group**. See Blackboard for date of submission. 

* The **filename** should be `lab2_id1_id2.ipynb`.

* The notebook is graded on a scale of **0-50**. The number of points for each question is indicated in parantheses. (Note: The total of 50 points is only for convenience and will be scaled- each of the four labs has the same weight in the course.) 

* The questions marked **BONUS** will give you bonus points (in addition to the 50 points); try them if you want an extra challenge. 

Notes on implementation:

* You should **write your code and answers in this iPython Notebook** (see http://ipython.org/notebook.html for reference material). If you have problems, please contact your teaching assistant.

* Use only **one cell for code** and **one cell for markdown** answers!    

    * Put all code in the cell with the `# YOUR CODE HERE` comment.
    
    * For theoretical question, put your solution in the YOUR ANSWER HERE cell.
    
* Test your code and **make sure we can run your notebook**

------

## Notation

$ \Sigma := \{ o_1, \dots, o_J \} $ is our set of **observations**

$\Lambda := \{c_1, \dots, c_K \}$ is our set of **state labels**

$\Sigma^*$ are **all possible sequences** of observations (including empty string $\epsilon$)

$\Lambda^*$ all possible sequences of hidden states (including empty string $\epsilon$)

> Extra info: we can say that $\Sigma^*$ is the [Kleene-closure](https://en.wikipedia.org/wiki/Kleene_star) of $\Sigma$, and $\Lambda^*$ the Kleene-closure of $\Lambda$.

## A simple example: The Baby HMM

We start with a simple example, so that we can easily verify that our code is correct.

Consider that we are modeling how a baby behaves. We observe the baby doing the following things: laughing (`laugh`), crying (`cry`), and sleeping (`sleep`). This is our set $\Sigma$ of **observations**. 

We presume that, at any moment, the baby can be either `hungry`, `bored`, or `happy`. Since babies cannot talk, each of these states is hidden. This is our set $\Lambda$ of **hidden states**.

**Now the question is: if we have a series of observations, can we predict what hidden states the baby went through?**

To tackle this problem, we assume that the baby behaves like a **1st order discrete Markov chain**: the baby's current state only depends on its previous hidden state. The baby can be described as an HMM. (Yay!)

For example, assume we observed the baby doing the following:

```
sleep cry laugh cry
cry cry laugh sleep
```

We will use these sequences as our **test set**. We can try to find out the states of the baby for those observations.

Now, to train our model, we will need some examples of **observations** and **states**; this is our **training set**:

```
laugh/happy cry/bored cry/hungry sleep/happy
cry/bored laugh/happy cry/happy sleep/happy
cry/hungry laugh/happy cry/bored sleep/happy
```

So we have **pairs** observation/state.

In [127]:
%matplotlib inline
import numpy as np
from collections import defaultdict
from collections import namedtuple

In [128]:
# read in test data
test_data = """sleep cry laugh cry
cry cry laugh sleep"""

def test_reader(test_lines):
    for line in test_lines.splitlines():
        yield line.split()

test_set = list(test_reader(test_data))

# read in train data
train_data = """laugh/happy cry/bored cry/hungry sleep/happy
cry/bored laugh/happy cry/happy sleep/bored
cry/hungry cry/bored sleep/happy"""

# for convenience, we define a Observation-State pair class
Pair = namedtuple("Pair", ["obs", "state"])
Pair.__repr__ = lambda x: x.obs + "/" + x.state

def train_reader(train_lines):
    for line in train_data.splitlines():
        # a pair is a string "observation/state" so we need to split on the "/"
        yield [Pair(*pair.split("/")) for pair in line.split()]

training_set = list(train_reader(train_data))

# print the results
print("test set (observations):")
for seq in test_set:
    print(seq)
print("\ntraining set (observation/state pairs):")
for seq in training_set:
    print(seq)

test set (observations):
['sleep', 'cry', 'laugh', 'cry']
['cry', 'cry', 'laugh', 'sleep']

training set (observation/state pairs):
[laugh/happy, cry/bored, cry/hungry, sleep/happy]
[cry/bored, laugh/happy, cry/happy, sleep/bored]
[cry/hungry, cry/bored, sleep/happy]


#### Vocabularies 
It's going to be very useful if we can map states and observations to integers, so that we can identify them by a number. If we don't do this, then our implementation will be much slower. (This is relevant when we work with larger data, e.g. for POS tagging)

Make sure you understand what is going on here: every time we look up a observation or state, the `defaultdict` will create a new key (index) if it has not seen that key (state or observation) before.

In [129]:
# create mappings from state/obs to an ID
state2i = defaultdict(lambda: len(state2i))
obs2i = defaultdict(lambda: len(obs2i))

for seq in training_set:
    for example in seq:
        state_id = state2i[example.state]
        obs_id = obs2i[example.obs]
        
print("\nOur vocabularies:")
print(state2i)
print(obs2i)


Our vocabularies:
defaultdict(<function <lambda> at 0x11f0b7d08>, {'happy': 0, 'bored': 1, 'hungry': 2})
defaultdict(<function <lambda> at 0x1104152f0>, {'laugh': 0, 'cry': 1, 'sleep': 2})


The HMM for the first training set sequence looks like this:

![hmm-baby-train-1](hmm-baby-train-1.png)

Now, we will **estimate** the following probability distributions:

- initial: $P( c_k \,|\, \texttt{start})$
- transition: $P( c_k \,|\, c_l )$
- final: $P(\texttt{stop} \,|\, c_k )$
- emission: $P( o_l \,|\, c_k)$

These distributions are all we need. Remember that:

- the probability of transitioning to a state $c_k$ only depends on one previous state $c_l$ (1st order Markov assumption).
- emitting an observation $o_l$ only depends on the state $c_k$.

## Finding the Maximum Likelihood Parameters

Now we would like to know what those distributions look like from our data. This is called **estimation**. Given our training data, we count how many times each event occurs and normalize the counts to form proper probability distributions. 

Let's first do counts for the start probabilities:

In [130]:
# we can get the number of states and observations from our dictionaries
num_states = len(state2i)
num_observations = len(obs2i)

# this creates a vector of length `num_states` filled with zeros
counts_start = np.zeros(num_states)

# now we count 1 every time a sequence starts with a certain state
# we look up the index for the state that we want to count using the `state2i` dictionary
for seq in training_set:
    counts_start[state2i[seq[0].state]] += 1.

print(counts_start)

[1. 1. 1.]


We see each state once at the beginning of a sequence in the training set, so that is why we have a count of 1 for each of them.

We now **normalize** those counts, so that we obtain a probability distribution:

In [131]:
# since p_start is a numpy object, we can call sum on it; easy!
total = counts_start.sum()

# normalize: divide each count by the total
p_start = counts_start / total  
print('start', '-->', p_start)

start --> [0.33333333 0.33333333 0.33333333]


We now turn to the **transition probabilities** and **stop probabilities**. We count, and then we normalize:

In [132]:
# we can transition from any state to any other state in principle,
# so we create a matrix filled with zeros so that we can count any pair of states
# in practice, some transitions might not occur in the training data
counts_trans = np.zeros([num_states, num_states])

# for the final/stop probabilities, we only need to store `num_states` values.
# so we use a vector
counts_stop = np.zeros(num_states)

# now we count transitions, one sequence at a time
for seq in training_set:
    for i in range(1, len(seq)):
        
        # convert the states to indexes
        prev_state = state2i[seq[i-1].state]
        current_state = state2i[seq[i].state]
        
        # count
        counts_trans[current_state, prev_state] += 1.

# count final states
for seq in training_set:
    state = state2i[seq[-1].state]
    counts_stop[state] += 1.

# print the counts
print("Transition counts:")
print(counts_trans)

print("Final counts:")
print(counts_stop)

Transition counts:
[[1. 2. 1.]
 [2. 0. 1.]
 [0. 1. 0.]]
Final counts:
[2. 1. 0.]


Now we can normalize again. We will need to collect the total counts per state.
Take some time to understand that the totals consist of the transition counts AND the final counts.

In [133]:
# Useful trick: np.sum(m, 0) sums matrix m along the first dimension:
print(counts_trans.sum(0))

[3. 3. 2.]


In [134]:
total_per_state = counts_trans.sum(0) + counts_stop
print("Total counts per state:\n", total_per_state, "\n")

# now we normalize
# here '/' works one column at a time in the matrix
p_trans = counts_trans / total_per_state
print("Transition probabilities:\n", p_trans)

# here '/' divides the values in each corresponding index in the 2 vectors
p_stop = counts_stop / total_per_state
print("Final probabilities:\n", p_stop, "\n")

Total counts per state:
 [5. 4. 2.] 

Transition probabilities:
 [[0.2  0.5  0.5 ]
 [0.4  0.   0.5 ]
 [0.   0.25 0.  ]]
Final probabilities:
 [0.4  0.25 0.  ] 



**So far so good!** Now let's take care of **emission probabilities**.

In [135]:
# now we create a matrix to keep track of emission counts
# in principle any states can emit any observation
# so we need a matrix again
counts_emiss = np.zeros([num_observations, num_states])

# count
for seq in training_set:
    for obs, state in seq:
        obs = obs2i[obs]
        state = state2i[state]
        counts_emiss[obs][state] += 1.

# normalize
p_emiss = counts_emiss / counts_emiss.sum(0)

print("emission counts:\n", counts_emiss)
print("p_emiss:\n", p_emiss)

emission counts:
 [[2. 0. 0.]
 [1. 3. 2.]
 [2. 1. 0.]]
p_emiss:
 [[0.4  0.   0.  ]
 [0.2  0.75 1.  ]
 [0.4  0.25 0.  ]]


This is a good moment for a sanity check. First, take a look at the training set to see if these probabilities are correct, i.e. check if  for each state $s_k$: $$\sum_l P(s_l \,|\, s_k) = 1.0$$ Note that this includes transitions to "stop" state, so you have to take those into account.

## Exercise 1: Sanity check (2.5 points)
Write a function `sanity_check(...)` that checks if all distributions are correct. If (and only if) it discovers an incorrect distribution, it should throw an **AssertionError**. 

If you want, you can include some print statements to see what is going on. 

**[Python hint]**: use python [assert](https://www.tutorialspoint.com/python/assertions_in_python.htm) statements.


In [136]:
def almost_one(p, eps=1e-3): # number is "small enough to be insignificant"*
    return (1.-eps) < p < (1. + eps)

# check for valid output
def sanity_check(p_start=None, p_trans=None, p_stop=None, p_emiss=None): # 'NONE' will be overwritten when calling*

    # check start distribution
    assert (almost_one(sum(p_start)) == 1), "Incorrect start distribution."
    
    # check transition and final distribution
    for previous_state in range(0, len(p_trans[0])):
        sum_p_trans = 0
        for current_state in range(0, len(p_trans)): 
            # print("probability P(previous|current):", p_trans[current_state][previous_state])
            sum_p_trans += p_trans[current_state][previous_state]
        assert (sum_p_trans + p_stop[previous_state] == 1), "Incorrect transition and final distribution."
        
    # check emission distribution
    for state in range(0, len(p_emiss[0])):
        sum_p_emiss = 0
        for observation in range(0, len(p_emiss)): 
            # print("probability P(observation|state):", p_emiss[observation][state])
            sum_p_emiss += p_emiss[observation][state]
        assert (sum_p_emiss == 1), "Incorrect emission distribution." 
        

In [137]:
# you can try your function out like this
# (do not use this cell for your solution)
try:
    sanity_check(p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
    print("All good!")
except AssertionError as e:
    print("There was a problem: %s" % str(e))

All good!


## Decoding a sequence

Ok, so we have estimated a model. Great. Now what? Well, now we can **decode**! 

Given an observation sequence $o_1, o_2, \dots, o_N$, we want to find the sequence of hidden states $s^* = s^*_1, s^*_2, \dots, s^*_N$ that **best** explains those observations.

But what does "best" mean?

1. If we are interested in the best **global** assignment of states to the sequence as a whole, we can use the **Viterbi** algorithm. 
2. If we care more about minimizing the **local** error of getting each $s_i$ right, we can use **posterior decoding** (also called *max marginal decoding*). *(This is a bonus exercise at the end!)*


### Decoding: The Viterbi Algorithm

Viterbi gives us the best global hidden state sequence, i.e.:

$$ \begin{array}{lll}
s^* &=& \arg\max_{s = s_1, s_2, \dots, s_N} P(s_1, s_2, \dots, s_N \,|\, o_1, o_2, \dots, o_N ) \\
    &=& \arg\max_{s = s_1, s_2, \dots, s_N} P(s_1, s_2, \dots, s_N, o_1, o_2, \dots, o_N ) 
\end{array}$$

To explain Viterbi we will make use of a **trellis**, a kind of graph that shows us the possible states for each time step. 

For our earlier example, the trellis looks like this:

![hmm-baby-train-1](hmm-baby-train-1-trellis.png)

Note, that we can now label the edges of this trellis with the following probabilities:

- $P_{\text start}(c_k \,|\, \text{start})$ on the three edges from "start"
- $P_{\text stop}(\text{stop} \,|\, c_k)$ on the three edges leading to "stop"
- $P_{\text trans}(c_k \,|\, c_{l})$ on each remaining edge from state $c_l$ to $c_k$
- $P_{\text emiss}(o \,|\, c_k)$ from each state $c_k$, to an observation $o$ made from that state (not shown here)

Do you see that our trellis nicely shows the independence assumptions of the HMM?

#### A Naive approach to get the best sequence

To see why the Viterbi algorithm is so useful, we can consider another way to calculate $s^*$:

- Iterate over all possible state sequences (all ways to go from `start` to `stop`)
    - Calculate the probability for that sequence
    - Store the highest probability seen so far and its sequence
- Return the sequence that had the maximum probability

The problem with this approach is that there are a lot of possible sequences!

# Exercise 2: How many possible state sequences are there? (5 points)

*This is a theoretical question.* Assume that you have a set $\Lambda$ of possible states (so there are $|\Lambda|$ states), and that the observation sequence is of length $N$. Assume there is a transition from any state to any other state.
Write down the formula that gives the number of sequences.

Since every transition can go through from any state to any other state,every possible next sequential state is individually and independently distributed. Thus, number of possible state squences should be |Λ|^N. 

#### The Viterbi algorithm

*We use a slightly different notation here compared to the lecture.*

So how do we find the path with the highest score? The idea is that we can use our trellis to represent an **exponential number of paths**. Since we are only interested in the highest-scoring path, for every state at every time step, we only need to keep track of the **highest** probability that can lead us to that state. We can disregard any other paths that lead to that state, since they will for sure not be part of the highest-scoring path.

Viterbi uses **dynamic programming**. Here, that means that we will re-use probabilities that we have already computed, so we never have to compute the score for the same sub-problem multiple times.

Let's start at the beginning.

For the first time step, the **viterbi score** is the transition probability of reaching a state $c_k$ from "start", times the probability of emitting the first observation $o_1$ from that state:

$$\text{viterbi}(1, c_k) = P_{\text start}( c_k \,|\, \text{start}) \times P_{\text emiss}(o_1 \,|\, c_k)$$

So, the Viterbi trellis represents the path with maximum probability in position $i$ when we are in state $y_i$ having observed $o_1, o_2, \dots, o_i$, the observations up to and including that point.

Now that we have the viterbi scores for all states of the first time step in our trellis, we can use the following **recursive formula** to get the scores for all other states, one time step at a time:

$$\text{viterbi}(i, c_k) = \big( \max_{c_l \in \Lambda} P_{\text trans}(c_k | c_l) \times \text{viterbi}(i-1, c_l) \big) \times P_{\text emiss}(o_i \,|\, c_k)$$

Finally, for our final state "stop" we need to do something special, since there is no observation there:

$$\text{viterbi}(N+1, \text{stop}) = \max_{c_l \in \Lambda} P_{\text stop}(\text{stop} \,|\, c_l) \times \text{viterbi}(i-1, c_l)  $$

This is all we need to know what probability the highest scoring path has! Do you see how the dynamic programming helps us to solve this task efficiently?

#### How did we get here?

Once we reach the "stop" state we know the maximum probability, but we forgot how we got there! If you don't see this immediately, remember that, whenever we computed the viterbi score for a state, we took the maximum over all previous states' viterbi scores, times the transition from those states. But we didn't keep track of which state was actually selected in that "max" operation. So now that we are in "stop", we don't know how we got there.

To solve this, we will use **backpointers**. Whenever we do a $\max$, we store what state was selected by that max (i.e. the $\arg\max$):

$$\text{backtrack}(i, c_k) = \arg\max_{c_l \in \Lambda} P_{\text trans}(c_k | c_l) \times \text{viterbi}(i-1, c_l)$$


#### Log probabilities

Now you know enough to implement Viterbi! But before we start.. Because probabilities tend to get rather small when multiplying, causing numerical instabilities, we will use **log probabilities**. This means that, instead of multiplying, we can now **sum** probabilities, because:

$$ \log(uv) = \log u + \log v$$

To get the probability of a  path trough our trellis from "start" to "stop", we can just **sum** the log-probabilities that we encounter. So, finding the best ("Viterbi") path means finding the path with the **highest score**.

# Exercise 3: convert all probabilities to log-probabilities (2.5 points)

In [138]:
# Note: only run this cell once; otherwise you get NaN values.
# If you get NaN values, try running all cells above first.
print("Before:")
print("\n Start:\n", p_start)
print("\n Trans:\n", p_trans)
print("\n Stop:\n", p_stop)
print("\n Emiss:\n", p_emiss, "\n")

def convert_to_log(p_start=None, p_trans=None, p_stop=None, p_emiss=None):
    """
    Convert all probabilities to log-probabilities
    
    Important: only run this function with normal probabilities as input! 
    If you run this twice, things will break.
    """    
    
    p_start = np.log(p_start)
    p_trans = np.log(p_trans)
    p_stop = np.log(p_stop)
    p_emiss = np.log(p_emiss)
    
    #raise NotImplementedError()
    return p_start, p_trans, p_stop, p_emiss

# do the conversion using your function
p_start, p_trans, p_stop, p_emiss = convert_to_log(p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)

print("After:")
print("\n Start:\n", p_start)
print("\n Trans:\n", p_trans)
print("\n Stop:\n", p_stop)
print("\n Emiss:\n", p_emiss)


Before:

 Start:
 [0.33333333 0.33333333 0.33333333]

 Trans:
 [[0.2  0.5  0.5 ]
 [0.4  0.   0.5 ]
 [0.   0.25 0.  ]]

 Stop:
 [0.4  0.25 0.  ]

 Emiss:
 [[0.4  0.   0.  ]
 [0.2  0.75 1.  ]
 [0.4  0.25 0.  ]] 

After:

 Start:
 [-1.09861229 -1.09861229 -1.09861229]

 Trans:
 [[-1.60943791 -0.69314718 -0.69314718]
 [-0.91629073        -inf -0.69314718]
 [       -inf -1.38629436        -inf]]

 Stop:
 [-0.91629073 -1.38629436        -inf]

 Emiss:
 [[-0.91629073        -inf        -inf]
 [-1.60943791 -0.28768207  0.        ]
 [-0.91629073 -1.38629436        -inf]]


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in log


#### Smoothing

Oops! We got a big red warning! What happened? 

Some probabilities were 0.0, and the log function is not defined for zero, resulting in a **warning**.

To prevent the error, we can add a small **smoothing** value to our **counts**, so that we never have a probability of zero.

To make things easier, we define a `normalize_all` function below that does all the normalization again,
but now adding a small value to all the counts.

In [139]:
def normalize(x, smoothing=0.1, axis=0):
    smoothed = x + smoothing
    return smoothed / smoothed.sum(axis)

def normalize_all(counts_start, counts_trans, counts_stop, counts_emiss, smoothing=0.1):
    """Normalize all counts to probabilities, optionally with smoothing."""
    p_start = normalize(counts_start, smoothing=smoothing)
    p_emiss = normalize(counts_emiss, smoothing=smoothing)
    
    counts_trans_smoothed = counts_trans + smoothing
    counts_stop_smoothed = counts_stop + smoothing
    total_trans_stop = counts_trans_smoothed.sum(0) + counts_stop_smoothed
    p_trans = counts_trans_smoothed / total_trans_stop
    p_stop = counts_stop_smoothed / total_trans_stop
    
    return p_start, p_trans, p_stop, p_emiss


# normalize with smoothing
smoothing = 0.1
p_start, p_trans, p_stop, p_emiss = normalize_all(
    counts_start, counts_trans, counts_stop, counts_emiss, smoothing=smoothing)

# convert to log-probabilities
print("Smoothed probabilities:\n", p_start, p_trans, p_stop, p_emiss, "\n")
p_start, p_trans, p_stop, p_emiss = convert_to_log(p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
print("Smoothed log-probabilities:\n", p_start, p_trans, p_stop, p_emiss)

Smoothed probabilities:
 [0.33333333 0.33333333 0.33333333] [[0.2037037  0.47727273 0.45833333]
 [0.38888889 0.02272727 0.45833333]
 [0.01851852 0.25       0.04166667]] [0.38888889 0.25       0.04166667] [[0.39622642 0.02325581 0.04347826]
 [0.20754717 0.72093023 0.91304348]
 [0.39622642 0.25581395 0.04347826]] 

Smoothed log-probabilities:
 [-1.09861229 -1.09861229 -1.09861229] [[-1.59108877 -0.7396672  -0.78015856]
 [-0.94446161 -3.78418963 -0.78015856]
 [-3.98898405 -1.38629436 -3.17805383]] [-0.94446161 -1.38629436 -3.17805383] [[-0.92576948 -3.76120012 -3.13549422]
 [-1.57239664 -0.32721291 -0.09097178]
 [-0.92576948 -1.36330484 -3.13549422]]


# Exercise 4: implement the Viterbi algorithm (40 points)

You will now implement the Viterbi algorithm. Complete the function `viterbi(sequence, p_start, p_trans, p_emiss, p_stop)` below.

**Input:** sequence ($o_1, ..., o_N$), $P_\text{start}$, $P_\text{trans}$, $P_\text{emiss}$, $P_\text{stop}$

*Forward pass: compute the best path for every end state*

- set $\text{viterbi}(1, c_k)$ for each $c_k$
- for $i=2$ to $N$, and for each $c_k$, set $\text{viterbi}(i, c_k$) and $\text{backtrack}(i, c_k)$
- $\text{max_prob} = \max_{c_l} P_{\text{stop}}(\text{stop} \,|\, c_l) \times viterbi(N, c_l)$

*Backward pass: backtrack to get most likely path*
- $\hat{s}_N = \arg\max_{c_l} P_\text{stop}(\text{stop} \,|\, c_l) \times viterbi(N, c_l)$
- for $i = N-1$ to $1$: $\hat{s}_i = \text{backtrack}(i+1, \hat{s}_{i+1})$

**Output:** max_prob, Viterbi path $\hat{s}_1, \hat{s}_2, \dots, \hat{s}_N$

In [140]:
# CELL VERWIJDEREN
num_states = len(p_start)

M = np.full([3,num_states], -np.inf)

# print(np.array(M))
for i in range(num_states):  
    M[0][i] =  p_start[i] + p_emiss[obs2i["sleep"]][i]

print("Start:\n", p_start)
print("\n             Happy     Bored       Hungry ")
print("Sleep ", p_emiss[list(obs2i.keys()).index("sleep")])
print("Hungry ", p_emiss[list(obs2i.keys()).index("laugh")])
print("Cry ", p_emiss[list(obs2i.keys()).index("cry")])

print("\n", M)


Start:
 [-1.09861229 -1.09861229 -1.09861229]

             Happy     Bored       Hungry 
Sleep  [-0.92576948 -1.36330484 -3.13549422]
Hungry  [-0.92576948 -3.76120012 -3.13549422]
Cry  [-1.57239664 -0.32721291 -0.09097178]

 [[-2.02438176 -2.46191713 -4.2341065 ]
 [       -inf        -inf        -inf]
 [       -inf        -inf        -inf]]


In [141]:
def viterbi(sequence, p_start=None, p_trans=None, p_stop=None, p_emiss=None):
    """
    Compute the Viterbi sequence. 
    Note: you have to use log-probabilities!
    
    The return value should be a tuple (max_prob, list_of_viterbi_states)
    Return:
      - best_score (float) the log-probability of the best path
      - best_path (int list) the best path as a list of state IDs
    """
    length = len(sequence)
    num_states = len(p_start)
    trellis = np.full([length, num_states], -np.inf) # matrix with length seq*num of states
    backpointers = -np.ones([length, num_states], dtype=int)
    
    # initialize probability of start states
    for i in range(num_states): 
            # fill in first column with all states**
            trellis[0][i] = p_start[i]+ p_emiss[obs2i[sequence[0]]][i]

   # creating the trellis scores 
    for t in range(1, length): # after t=0, each time t in the sequence
        for i in range(num_states):           # each state 
            max_tr_prob = trellis[t-1][0] + p_trans[i][0] # assign the first transition probability to indexes 
            prev_st_selected = 0 
            for j in range(1,num_states): # calling the previous state in time t
                tr_prob = trellis[t-1][j] + p_trans[i][j]
                if tr_prob > max_tr_prob:
                    max_tr_prob = tr_prob 
                    prev_st_selected = j 
                  
            
            max_prob = max_tr_prob + p_emiss[obs2i[sequence[t]]][i]           
            trellis[t][i] = max_prob
        
            backpointers[t][i] = prev_st_selected # backpointers are assigned with index
     
    for i in range(num_states): # adding the last probability
            trellis[length - 1][i] += p_stop[i]
    
    best_score = np.max(trellis[-1][:])
             
    print("trellis: \n", trellis)
    print("shape trellis: \n", np.shape(trellis))
    print("backpointers: \n", backpointers)   
   
    # trellis to store Viterbi scores
    # we store -inf as our initial scores since log(0)=-inf
    
     # back-tracking
    best_path = np.zeros(length).astype(int)
    best_path[length-1] =  np.argmax(trellis[length-1,:]) # last state
    for i in range(length - 2,1,-1): # states of (last-1)th to 0th time step
        best_path[i-1] = backpointers[i,best_path[i]]
        
    # backpointers to backtrack (to remember what prev. state caused the maximum score)
    # we initialize with -1 values, to represent a non-existing index

    return best_score, best_path

#### Trying out Viterbi

Once you have implemented the Viterbi algorithm, try it out on the following sequence.

Note: to get all points, make sure that the cell below runs.

In [142]:
test_sequence = test_set[0]
print(test_set[0])

['sleep', 'cry', 'laugh', 'cry']


In [143]:
"""Test out your Viterbi-algorithm here."""

test_sequence_1 = test_set[0]
test_sequence_2 = test_set[1]

print("First test\n")
best_score_1, best_path_1 = viterbi(test_sequence_1, p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
print("Test sequence is:", test_set[0])
print("The log-probability of the best path is:", best_score_1)
print("The best path as a list of state IDs is:", best_path_1, "\n")


print("Second test \n")
best_score_2, best_path_2 = viterbi(test_sequence_2, p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
print("Test sequence is:", test_set[1])
print("The log-probability of the best path is:", best_score_2)
print("The best path as a list of state IDs is:", best_path_2)

i2state = {v : k for k, v in state2i.items()}
# print([i2state[i] for i in best_path])

First test

trellis: 
 [[ -2.02438176  -2.46191713  -4.2341065 ]
 [ -4.77398097  -3.29605628  -3.93918327]
 [ -4.96149296  -8.48054194  -7.81784486]
 [ -9.06943998  -7.61946184 -12.21950261]]
shape trellis: 
 (4, 3)
backpointers: 
 [[-1 -1 -1]
 [ 1  0  1]
 [ 1  2  1]
 [ 0  0  0]]
Test sequence is: ['sleep', 'cry', 'laugh', 'cry']
The log-probability of the best path is: -7.619461837738772
The best path as a list of state IDs is: [0 1 0 1] 

Second test 

trellis: 
 [[ -2.67100893  -1.4258252   -1.18958407]
 [ -3.54213927  -2.29695554  -2.90309134]
 [ -3.96239221  -7.44445001  -6.81874411]
 [ -7.42371207  -7.65645302 -14.2649243 ]]
shape trellis: 
 (4, 3)
backpointers: 
 [[-1 -1 -1]
 [ 2  2  1]
 [ 1  2  1]
 [ 0  0  0]]
Test sequence is: ['cry', 'cry', 'laugh', 'sleep']
The log-probability of the best path is: -7.423712066090819
The best path as a list of state IDs is: [0 1 0 0]


# Congratulations!

You have reached the end of lab 2.

If you want an additional challenge, and to attempt 10 additional (bonus) points, go ahead and try the following sections. Otherwise, you're done!

# Bonus: Posterior Decoding

What if we don't care about the global sequence of hidden states, but more about getting **individual** states right? 
We now aim to find the state with the **highest state posterior** for each position. So what is a state posterior? It is defined as:

$$ P( s_i \,|\, o_1, o_2, \dots, o_N) $$

It gives the probability, with observation sequence $o_1, o_2, \dots, o_N$, that the i'th hidden state was $s_i$. 

The best state $s_i^*$ for that position is then:

$$ s_i^* = \arg\max_{s_i \in \Lambda} P( s_i \,|\, o_1, o_2, \dots, o_N) $$

If we calculate this for each position, we are performing posterior decoding. Do you see the difference with Viterbi? Now we choose the hidden states **independenly**, based on the observations $o = o_1, o_2, \dots, o_N$, one position at a time. 

#### How to calculate a state posterior?

Good question. Calculating the state posterior is not so easy. Remember that we are interested in one specific time step. To choose the best state for this time step, we need to take into account all paths that lead there. You will soon see why. 

First, let's define a few useful terms.

**Sequence posterior**
$$ P( s = s_1, s_2, \dots, s_N \,|\, o = o_1, o_2, \dots, o_N)  = \frac{P(s, o)}{P(o)}$$

To compute this, we need the **likelihood**:

$$ P( o = o_1, o_2, \dots, o_N ) = \sum_{s} P(o, s)$$

To calculate the likelihood, we thus need to sum over **all possible state sequences s**!

Since the number of state sequences can grow so quickly, we will do something smarter than summing over all of them.
(We used a similar trick with the Viterbi-algorithm!)

#### Forward and Backward probabilities

We can use compute **forward** and **backward** probabilities, which help us compute the likelihood in **linear time**: O(N).

Let's first see how they are computed, before we will use them to calculate the likelihood (and finally the state posterior).

The forward probability for a position i and a state $c_k$, gives us the probability of being in that state and that position, having observed $o_1, o_2, \dots, o_i$:

$$\text{forward}(i, c_k) = P(c_k, o_1, o_2, \dots, o_i)$$

Because of the independence assumptions in the HMM, we can calculate the forward probability of position $i$ using the forward probabilities of each state in position $i-1$:

$$\text{forward}(i, c_k) = \Big( \sum_{c_l \in \Lambda} P_{\text trans}(c_k \,|\, c_l) \times \text{forward}(i-1, c_l) \Big) \times P_{\text emiss}(o_i \,|\, c_k)  $$

And our special cases:

$$\text{forward}(1, c_k) =P_{\text start}(c_k \,|\, \text{start}) \times P_{\text emiss} (o_1 \, |\, c_k)$$
$$\text{forward}(N+1, \text{stop}) = \sum_{c_l \in \Lambda} P_{\text trans}(\text{stop} \,|\, c_l) \times \text{forward}(N, c_l) $$

Did you notice that this is almost exactly the same as the Viterbi scores? Instead of taking the maximum, we are now **summing**!

**Warning: you cannot "just sum" two probabilities in the log-domain!**
A brute-force way to do this correctly would be to do $\log(\exp(a) + \exp(b)$, i.e. convert the probabilities back and then sum, and then convert them to the log domain again. But this exposes us to numeric instabilities again! So a better way to do it is using $$\log(\exp(a) + \exp(b)) = a + \log(1 + \exp(b − a)) \qquad \text{for } a < b$$

We are providing a function `logsum()` for you that sums a list of values in the log-domain (using the above strategy).

Now that we can compute forward probabilities: did you notice that $\text{forward}(N+1, \text{stop})$ gives us the **likelihood**, i.e. $P(o_1, o_2, \dots, o_N)$? Take a moment to see why.

Sadly, this is still not enough to calculate the state posteror. (You probably guessed, since this is calleed the Forward-**Backward** algorithm!). Right now we know the probability of being in state $s_i$ having observed $o_1, o_2, \dots, o_i$, but we do **not** know the probability of being in that state knowing $o_{i+1}, \dots, o_N$. This is what the backward probability gives us:

$$\text{backward}(i, c_l) = \sum_{c_k \in \Lambda} P_\text{trans}(c_k \,|\, c_l) \times \text{backward}(i+1,c_k) \times P_\text{emiss}(o_{i+1} \,|\, c_k)$$

And our special cases:

$$\text{backward}(N, c_l) = P_\text{stop}( \text{stop} \,|\, c_l)$$
$$\text{backward}(0, \text{start}) = \sum_{c_k \in \Lambda} P_{\text start}(c_k| \text{start}) \times \text{backward}(1, c_k) \times P_{\text emiss}(o_1 \,|\, c_k)$$

Can you see that we can also calculate the likelihood using just backward probabilities? You can find it in $\text{backward}(0, \text{start})$.


Now, implement the forward and backward algorithms.

The forward function should return two values:

- all forward scores for the trellis (length x num_states)
- the forward score of the stop state (not part of the trellis)

The backward function should similarly return two values.

In [201]:
def logsum(logv):
    """Sum of probabilities in log-domain."""
    res = -np.inf
    for val in logv:
        res = logsum_pair(res, val)
    return res

def logsum_pair(logx, logy):
    """
    Return log(x+y), avoiding arithmetic underflow/overflow.
    """
    if logx == -np.inf:
        return logy
    elif logx > logy:
        return logx + np.log1p(np.exp(logy-logx))
    else:
        return logy + np.log1p(np.exp(logx-logy))

In [202]:
def forward(sequence, p_start=None, p_trans=None, p_stop=None, p_emiss=None):
    """
    Compute Forward probabilities.
    Note: all probabilities should be log-probabilities.
    Return:
      - trellis with forward probabilities, excluding the "stop" cell
      - the forward probability of the stop cell (this is the log-likelihood!)
    """
    
    length = len(sequence)
    num_states = len(p_start)
    
    # trellis to store forward probabilities
    fw_trellis = np.full([length, num_states], -np.inf)
    f_prev = np.full(num_states, -np.inf)
    
    
    # base case: P(start|state) * P(obs|state)
    for i in range(num_states): 
        fw_trellis[0][i] = p_start[i] + p_emiss[obs2i[sequence[0]]][i]

 
    # inductive step:
    for t in range(1, length): 
        
        # states @ t
        for i in range(num_states): 
            f_prev_sum = np.full(num_states, -np.inf)
            
            # states @ t-1
            for j in range(num_states): 
                
                # store: P(previous st) * P(current st|previous st)
                f_prev_sum[j] = (fw_trellis[t-1][j] + p_trans[i][j]) 

            # current st = sum(P(state|previous) * P(previous)) * P(obs|state)
            fw_trellis[t][i] = logsum(f_prev_sum) + p_emiss[obs2i[sequence[0]]][i]
    
    
    # final step: P(end|state) * P(state|previous)
    f_termination = np.full(num_states, -np.inf)
    
    for st in range(num_states):
        f_termination[st] = p_stop[st] + fw_trellis[length-1][st]
        
    f_log_likelihood = logsum(f_termination)
            

    return fw_trellis, f_log_likelihood


In [203]:
# test forward algo.
test_sequence = test_set[0]
print("Test seq == ", test_set[0], "\n")
fw_trellis, f_log_likelihood = forward(test_sequence, p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
print("fw_trellis:\n ", fw_trellis, "\n")
print("f_log_likelihood:\n", f_log_likelihood)


Test seq ==  ['sleep', 'cry', 'laugh', 'cry'] 

fw_trellis:
  [[ -2.02438176  -2.46191713  -4.2341065 ]
 [ -3.52616174  -4.17766301  -6.84999773]
 [ -5.20909492  -5.76355457  -8.55654658]
 [ -6.84016071  -7.44451956 -10.15504388]] 

f_log_likelihood:
 -7.480698080206542


In [204]:
def backward(sequence, p_start=None, p_trans=None, p_stop=None, p_emiss=None):
    """
    Compute Backward probabilities.
    Note: all probabilities should be log-probabilities.  Return:
      - trellis with backward probabilities, excluding the "start" cell
      - the forward probability of the start cell (this is ALSO the log-likelihood!)
    """
    
    length = len(sequence)
    num_states = len(p_start)
    
    # trellis to store forward probabilities
    bw_trellis = np.full([length, num_states], -np.inf)
    
    # base case: P(end|state) == 1
    for i in range(num_states): 
        bw_trellis[length-1][i] = 1
       
    
    # inductive step:
    for t in range(1, length): 
        
        # states @ t
        for i in range(num_states): 
            b_next_sum = np.full(num_states, -np.inf)
            
            # states @ t+1
            for j in range(num_states): 
                
                # store: P(next st) * P(next st|previous st) * P(obs|next st)
                b_next_sum[j] = (bw_trellis[length-t][j] + p_trans[i][j]) + p_emiss[obs2i[sequence[0]]][j]

            # current st = sum(P(next st) * P(previous st|next st) * P(obs|next st))
            bw_trellis[length-1-t][i] = logsum(b_next_sum) 
 

    # final step: sum(P(start|state) * P(obs|state) * P(state))
    termination = np.full(num_states, -np.inf)
    
    for st in range(num_states):
        termination[st] = p_start[st] + p_emiss[obs2i[sequence[0]]][st] + bw_trellis[0][st]
        
    b_log_likelihood = logsum(termination)
            
        
    return bw_trellis, b_log_likelihood


In [205]:
# test backward algo.
test_sequence = test_set[0]
print("Test seq == ", test_set[0], "\n")
bw_trellis, b_log_likelihood = backward(test_sequence, p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
print("bw_trellis:\n ", bw_trellis, "\n")
print("b_log_likelihood:\n", b_log_likelihood)


Test seq ==  ['sleep', 'cry', 'laugh', 'cry'] 

bw_trellis:
  [[-3.81577894 -3.98260243 -4.92093738]
 [-2.18471316 -2.30163744 -3.32244007]
 [-0.50177997 -0.71574588 -1.61589123]
 [ 1.          1.          1.        ]] 

b_log_likelihood:
 -5.380988687608448


In [206]:
def forward_backward(sequence):
    """
    Compute forward and backward probabilities.
    Return:
    - fw_trellis
    - fw_log_likelihood (the value of the "stop" cell, not part of the trellis)
    - bw_trellis
    - bw_log_likelihood (the value of the "start" cell, not part of the trellis)
    """
    fw_trellis, fw_ll = forward(sequence, p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
    bw_trellis, bw_ll = backward(sequence, p_start=p_start, p_trans=p_trans, p_stop=p_stop, p_emiss=p_emiss)
    return fw_trellis, fw_ll, bw_trellis, bw_ll

In [207]:
test_sequence = test_set[0]
fw_trellis, fw_ll, bw_trellis, bw_ll = forward_backward(test_sequence)

print("Test sequence: \n", test_sequence, "\n")
print("Forward trellis: \n", fw_trellis)
print("Forward log-likelihood: \n", fw_ll, "\n")
print("Backward trellis: \n", bw_trellis)
print("Bakcward log-likelihood: \n", bw_ll)

Test sequence: 
 ['sleep', 'cry', 'laugh', 'cry'] 

Forward trellis: 
 [[ -2.02438176  -2.46191713  -4.2341065 ]
 [ -3.52616174  -4.17766301  -6.84999773]
 [ -5.20909492  -5.76355457  -8.55654658]
 [ -6.84016071  -7.44451956 -10.15504388]]
Forward log-likelihood: 
 -7.480698080206542 

Backward trellis: 
 [[-3.81577894 -3.98260243 -4.92093738]
 [-2.18471316 -2.30163744 -3.32244007]
 [-0.50177997 -0.71574588 -1.61589123]
 [ 1.          1.          1.        ]]
Bakcward log-likelihood: 
 -5.380988687608448


## Posterior Decoding

Implement a function that, given the forward and backward probabilities, and a sequence of observations $o_1, o_2, \dots, o_N$, returns the best hidden state sequence, according to posterior decoding.

In [208]:
def posterior_decode(sequence, fw_trellis, bw_trellis, ll, p_trans, p_emiss):
    """
    Return best hidden state sequence according to Posterior decoding
    """

    length = len(sequence)
    num_states = fw_trellis.shape[1]
        
    # calculate the state posteriors
    state_posteriors = np.zeros([length, num_states])
    
    for i in range(length):
        for st in range(num_states):
            state_posteriors[i][st] = fw_trellis[i][st] * bw_trellis[i][st] / (p_stop[st] + fw_trellis[length-1][st])
            
    state_posteriors = np.exp(state_posteriors)
    
    # the best states are simply the arg max of the state posteriors
    best_sequence = np.argmax(state_posteriors, axis=1)

    return state_posteriors, best_sequence

In [209]:
state_posteriors, best_sequence = posterior_decode(test_sequence, fw_trellis, bw_trellis, fw_ll, p_trans, p_emiss)

print(state_posteriors)
print(best_sequence)


[[0.37072721 0.32946069 0.20956725]
 [0.37172584 0.33660207 0.18142133]
 [0.71478961 0.62679134 0.35451594]
 [2.40770976 2.32336425 2.14178962]]
[0 0 0 0]


              Acknowlegements: A version of this lab was originally developed in collaboration with Joost Bastings